In [1]:
pip install pydantic_ai -q

Note: you may need to restart the kernel to use updated packages.


c:\Users\pavel\Desktop\Bicocca_DS_2026\.venv\Scripts\python.exe: No module named pip


In [ ]:
import os
from pydantic_ai.providers.openai import OpenAIProvider
from pydantic_ai import Agent
from typing import Optional
from typing import Union
from pydantic_ai.models.openai import OpenAIResponsesModel
from pydantic import BaseModel, Field
import pandas as pd
import pickle
import re
import json
from collections import defaultdict
from typing import Dict, List
import asyncio


from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
model_5_responses = OpenAIResponsesModel(
    "gpt-5", provider=OpenAIProvider(api_key=os.getenv("OPENAI_API_KEY"))
)

In [ ]:
information_test_system_prompt = """#Role
You are an expert in extracting specific information from text.
Your task is to evaluate it the source is valuable 0 - no information related found. 0.5 - if there is information but not full/not correct. 1 - full information is found.
"""

In [ ]:
from typing import Literal


class GradeAnswerFull(BaseModel):
    """Score indicating how well the content helps answer the question."""

    relevant_information: Literal[0, 0.5, 1] = Field(
        description=(
            "Relevance score: "
            "0 = completely irrelevant; "
            "0.5 = some relevant information present but incomplete or incorrect; "
            "1 = fully relevant and helps answer the question"
        )
    )

In [ ]:
information_test_agent = Agent(
    model_5_responses,
    model_settings={"temperature": 0.0},
    system_prompt=information_test_system_prompt,
    output_type=GradeAnswerFull,
)

In [ ]:
ground_truth = pd.read_csv(
    "C:/Users/pavel/Desktop/Bicocca_DS_2026/app/data/ground_truth.csv"
)

In [ ]:
def get_ground_truth_value(company_name, column_name):
    """Extract value by company name and column name."""
    try:
        return ground_truth.loc[
            ground_truth["company"] == company_name, column_name
        ].iloc[0]
    except (IndexError, KeyError):
        return None

In [ ]:
def ddict():
    return defaultdict(dict)

### Brave

In [ ]:
with open("missing_results/search_results_brave_v2.pkl", "rb") as f:
    brave = pickle.load(f)

In [ ]:
with open("missing_results/search_results_duckduckgo_v2.pkl", "rb") as f:
    duck = pickle.load(f)

In [ ]:
with open("missing_results/search_results_exa.pkl", "rb") as f:
    exa = pickle.load(f)

In [ ]:
with open("missing_results/search_results_jina.pkl", "rb") as f:
    jina = pickle.load(f)

In [ ]:
with open("missing_results/search_results_tavily_v2.pkl", "rb") as f:
    tavily = pickle.load(f)

In [ ]:
with open("missing_results/search_results_you.pkl", "rb") as f:
    you = pickle.load(f)

In [ ]:
import pandas as pd
import json
from collections import defaultdict


def convert_brave_to_dataframe(data_dict):
    """
    Convert nested defaultdict structure to DataFrame.

    Parameters:
    -----------
    data_dict : defaultdict
        Nested structure: {company: {metric: {'tool_response': '[...]'}}}

    Returns:
    --------
    pd.DataFrame with columns: company, query, title, snippet, url
    """
    rows = []

    for company, metrics_dict in data_dict.items():
        for query, query_data in metrics_dict.items():
            # Skip if not a dict or missing tool_response
            if not isinstance(query_data, dict):
                continue

            tool_response = query_data.get("tool_response", "[]")

            # Skip if empty
            if not tool_response or tool_response == "[]":
                continue

            try:
                # Parse JSON string to list
                responses = json.loads(tool_response)

                # Ensure responses is a list
                if not isinstance(responses, list):
                    continue

                # Extract each response
                for response in responses:
                    if isinstance(response, dict):
                        rows.append(
                            {
                                "company": company,
                                "query": query,
                                "title": response.get("title", ""),
                                "snippet": response.get("snippet", ""),
                                "url": response.get(
                                    "link", ""
                                ),  # 'link' field becomes 'url' column
                            }
                        )
            except (json.JSONDecodeError, TypeError, AttributeError) as e:
                print(f"Error processing {company} - {query}: {e}")
                continue

    # Create DataFrame
    df = pd.DataFrame(rows)

    return df


brave_df = convert_brave_to_dataframe(brave)
brave_df["engine"] = "brave"

In [ ]:
import pandas as pd
from collections import defaultdict


def convert_duck_to_dataframe(data_dict, include_res_num=False):
    """
    Convert nested defaultdict structure to DataFrame.

    Parameters:
    -----------
    data_dict : defaultdict
        Structure: {company: {query: [{'title': ..., 'snippet': ..., 'link': ...}, ...]}}
    include_res_num : bool
        If True, adds a 'res_num' column (0-indexed) for each result within company-query

    Returns:
    --------
    pd.DataFrame with columns: company, query, title, snippet, url (and optionally res_num)
    """
    rows = []

    for company, queries_dict in data_dict.items():
        for query, results_list in queries_dict.items():
            # Ensure results_list is a list
            if not isinstance(results_list, list):
                continue

            # Extract each result with optional res_num
            for res_num, result in enumerate(results_list):
                if isinstance(result, dict):
                    row = {
                        "company": company,
                        "query": query,
                        "title": result.get("title", ""),
                        "snippet": result.get("snippet", ""),
                        "url": result.get("link", ""),  # 'link' becomes 'url'
                    }
                    if include_res_num:
                        row["res_num"] = res_num
                    rows.append(row)

    # Create DataFrame
    df = pd.DataFrame(rows)

    return df

In [ ]:
duck_df = convert_duck_to_dataframe(duck)
duck_df["engine"] = "duckduckgo"

In [ ]:
import pandas as pd
from collections import defaultdict


def convert_exa_to_dataframe(
    data_dict, include_extra_fields=False, max_snippet_length=500
):
    """
    Convert nested defaultdict with SearchResponse objects to DataFrame.

    Parameters:
    -----------
    data_dict : defaultdict
        Structure: {company: {query: SearchResponse(results=[Result(...), ...])}}
    include_extra_fields : bool
        If True, includes additional fields like author, published_date, etc.
    max_snippet_length : int
        Maximum length for snippet text (default: 500)

    Returns:
    --------
    pd.DataFrame with columns: company, query, title, snippet, url (and optionally more)
    """
    rows = []

    for company, queries_dict in data_dict.items():
        for query, search_response in queries_dict.items():
            # Check if it's a SearchResponse object with results attribute
            if hasattr(search_response, "results"):
                results_list = search_response.results
            elif isinstance(search_response, list):
                results_list = search_response
            else:
                continue

            # Extract each result
            for result in results_list:
                # Get snippet from text or summary
                snippet = ""
                if hasattr(result, "summary") and result.summary:
                    snippet = result.summary
                elif hasattr(result, "text") and result.text:
                    snippet = result.text[:max_snippet_length]
                    if len(result.text) > max_snippet_length:
                        snippet += "..."

                row = {
                    "company": company,
                    "query": query,
                    "title": getattr(result, "title", ""),
                    "snippet": snippet,
                    "url": getattr(result, "url", ""),
                }

                # Add extra fields if requested
                if include_extra_fields:
                    row["author"] = getattr(result, "author", "")
                    row["published_date"] = getattr(result, "published_date", None)
                    row["score"] = getattr(result, "score", None)

                rows.append(row)

    # Create DataFrame
    df = pd.DataFrame(rows)

    return df


# Usage:
# df = convert_searchresponse_to_dataframe(brave, include_extra_fields=True)

In [ ]:
exa_df = convert_exa_to_dataframe(exa)
exa_df["engine"] = "exa"

In [ ]:
def convert_jina_to_dataframe(data_dict, company=None):
    """
    Convert Jina search results structure to DataFrame.

    Parameters:
    -----------
    data_dict : dict or defaultdict
        Structure: {query: [{'title': ..., 'link': ..., 'snippet': ..., 'content': ...}, ...]}
        OR {company: {query: [{'title': ..., 'link': ..., 'snippet': ..., 'content': ...}, ...]}}
    company : str, optional
        Company name if data_dict is at query level. If None and structure has company level, will extract from structure.

    Returns:
    --------
    pd.DataFrame with columns: company, query, title, snippet, url
    """
    rows = []

    # Check if structure has company level
    if company is None:
        # Check first key to see if it's a company name or query
        first_key = next(iter(data_dict.keys())) if data_dict else None
        if first_key and isinstance(data_dict[first_key], dict):
            # Structure: {company: {query: [...]}}
            for comp, queries_dict in data_dict.items():
                for query, results_list in queries_dict.items():
                    if not isinstance(results_list, list):
                        continue
                    for result in results_list:
                        if isinstance(result, dict):
                            rows.append(
                                {
                                    "company": comp,
                                    "query": query,
                                    "title": result.get("title", ""),
                                    "snippet": result.get("snippet", ""),
                                    "url": result.get(
                                        "link", ""
                                    ),  # 'link' becomes 'url'
                                }
                            )
        else:
            # Structure: {query: [...]} - need company parameter
            if company is None:
                raise ValueError(
                    "If data_dict is at query level, 'company' parameter must be provided"
                )
            for query, results_list in data_dict.items():
                if not isinstance(results_list, list):
                    continue
                for result in results_list:
                    if isinstance(result, dict):
                        rows.append(
                            {
                                "company": company,
                                "query": query,
                                "title": result.get("title", ""),
                                "snippet": result.get("snippet", ""),
                                "url": result.get("link", ""),  # 'link' becomes 'url'
                            }
                        )
    else:
        # Company provided, structure is {query: [...]}
        for query, results_list in data_dict.items():
            if not isinstance(results_list, list):
                continue
            for result in results_list:
                if isinstance(result, dict):
                    rows.append(
                        {
                            "company": company,
                            "query": query,
                            "title": result.get("title", ""),
                            "snippet": result.get("snippet", ""),
                            "url": result.get("link", ""),  # 'link' becomes 'url'
                        }
                    )

    # Create DataFrame
    df = pd.DataFrame(rows)

    return df

In [ ]:
jina_df = convert_jina_to_dataframe(jina)
jina_df["engine"] = "jina"

In [ ]:
def convert_tavily_to_dataframe(data_dict, max_snippet_length=500):
    """
    Convert Jina search results structure to DataFrame.

    Parameters:
    -----------
    data_dict : dict or defaultdict
        Structure: {company: {query: [{'url': ..., 'title': ..., 'content': ..., 'score': ...}, ...]}}
    max_snippet_length : int
        Maximum length for snippet text extracted from content (default: 500)

    Returns:
    --------
    pd.DataFrame with columns: company, query, title, snippet, url
    """
    rows = []

    for company, queries_dict in data_dict.items():
        for query, results_list in queries_dict.items():
            # Ensure results_list is a list
            if not isinstance(results_list, list):
                continue

            # Extract each result
            for result in results_list:
                if isinstance(result, dict):
                    # Get snippet from content
                    content = result.get("content", "")
                    snippet = content[:max_snippet_length] if content else ""
                    if content and len(content) > max_snippet_length:
                        snippet += "..."

                    rows.append(
                        {
                            "company": company,
                            "query": query,
                            "title": result.get("title", ""),
                            "snippet": snippet,
                            "url": result.get("url", ""),
                        }
                    )

    # Create DataFrame
    df = pd.DataFrame(rows)

    return df

In [ ]:
tavily_df = convert_tavily_to_dataframe(tavily)
tavily_df["engine"] = "tavily"

In [ ]:
import pandas as pd


def convert_you_to_dataframe(data_dict, company=None):
    """
    Convert You search results structure to DataFrame.

    Parameters:
    -----------
    data_dict : dict or defaultdict
        Structure:
        {company: {query: {"web": [ {url, title, description, snippets, ...}, ... ]}}}
        OR
        {query: {"web": [ {url, title, description, snippets, ...}, ... ]}}
    company : str, optional
        Company name if data_dict is at query level.
        If None and structure has company level, company will be inferred.

    Returns:
    --------
    pd.DataFrame with columns: company, query, title, snippet, url
    """
    rows = []

    # Detect whether company level exists
    if company is None:
        first_key = next(iter(data_dict.keys()), None)

        if (
            first_key
            and isinstance(data_dict[first_key], dict)
            and "web" not in data_dict[first_key]
        ):
            # Structure: {company: {query: {"web": [...]}}}
            for comp, queries_dict in data_dict.items():
                for query, source_dict in queries_dict.items():
                    web_results = source_dict.get("web", [])
                    if not isinstance(web_results, list):
                        continue

                    for result in web_results:
                        if isinstance(result, dict):
                            rows.append(
                                {
                                    "company": comp,
                                    "query": query,
                                    "title": result.get("title", ""),
                                    "snippet": result.get("description")
                                    or " ".join(result.get("snippets", [])),
                                    "url": result.get("url", ""),
                                }
                            )
        else:
            # Structure: {query: {"web": [...]}} → company must be provided
            raise ValueError(
                "If data_dict is at query level, 'company' parameter must be provided"
            )
    else:
        # Company provided, structure is {query: {"web": [...]}}
        for query, source_dict in data_dict.items():
            web_results = source_dict.get("web", [])
            if not isinstance(web_results, list):
                continue

            for result in web_results:
                if isinstance(result, dict):
                    rows.append(
                        {
                            "company": company,
                            "query": query,
                            "title": result.get("title", ""),
                            "snippet": result.get("description")
                            or " ".join(result.get("snippets", [])),
                            "url": result.get("url", ""),
                        }
                    )

    return pd.DataFrame(rows)

In [ ]:
you_df = convert_tavily_to_dataframe(tavily)
you_df["engine"] = "you"

In [66]:
you_df

,company,query,title,snippet,url,engine
0,Adyen,CEO in 2024,Adyen - Wikipedia,| Company type | Naamloze vennootschap |\n| Tr...,https://en.wikipedia.org/wiki/Adyen,you
1,Adyen,CEO in 2024,Adyen N.V. (ADYENA) Leadership & Management Te...,Adyen N.V.\n\nBATS-CHIXE:ADYENA Stock Report\n...,https://simplywall.st/stocks/gb/diversified-fi...,you
2,Adyen,CEO in 2024,Money 20/20 2024 is in the books . Ingo Uytdeh...,"# Adyen’s Post\n\n351,835 followers\n\n Report...",https://www.linkedin.com/posts/adyen_money-202...,you
3,Adyen,CEO in 2024,"Ingo Uytdehaage, Adyen | All-In Summit 2024 - ...","# Ingo Uytdehaage, Adyen | All-In Summit 2024\...",https://www.youtube.com/watch?v=4ZX8XlTPJrs,you
4,Adyen,CEO in 2024,Lifetime of Achievement: Pieter Van Der Does |...,"Since 2007, Adyen has maintained consistent do...",https://fintechmagazine.com/articles/lifetime-...,you
...,...,...,...,...,...,...
595,Zalando,gross margin in 2024,[PDF] Zalando group financials as of Q4 2024,23.1% 29.7% (0.1%) (1.9%) (0.6%) 3.4% 1.5% 5.0...,https://corporate.zalando.com/sites/default/fi...,you
596,Zalando,gross margin in 2024,[PDF] Download Version Website Full Year Publi...,GMV 2 Accumulated proﬁt contribution before ma...,https://corporate.zalando.com/sites/default/fi...,you
597,Zalando,gross margin in 2024,ZAL - Zalando SE Key Metrics - Morningstar,"| Income Statement (in Bil, except ratios) | ...",https://www.morningstar.com/stocks/xhan/zal/ke...,you
598,Zalando,gross margin in 2024,Zalando-SE_AGM-2025_1_Annual-report_2024.pdf,1 Company 2 Combined management report 3 Conso...,https://corporate.zalando.com/sites/default/fi...,you


In [ ]:
all_searches = pd.concat([brave_df, duck_df, exa_df, jina_df, tavily_df, you_df])

all_searches["search_num"] = all_searches.groupby(
    ["engine", "company", "query"]
).cumcount()

In [ ]:
calculated_searches = pd.read_csv("data/searches_results.csv")

In [ ]:
calculated_searches["pos"] = range(len(calculated_searches))

In [ ]:
all_searches["pos"] = range(len(all_searches))

In [ ]:
you_searches = all_searches[all_searches["engine"] == "you"]

positions = you_searches["pos"].tolist()


In [ ]:
matching = calculated_searches.merge(
    you_searches[["company", "query", "title", "snippet", "url"]],
    on=["company", "query", "title", "snippet", "url"],
    how="inner",
)
matching


,company,query,title,snippet,url,engine,search_num,score,pos
0,Adyen,CEO in 2024,10 Adyen Executives & Org Chart - Clay,"Date\n\nNovember 15, 2024\n\nIndex\n\nTOC Head...",https://www.clay.com/dossier/adyen-executives,tavily,6,0.5,2
1,Adyen,CEO in 2024,10 Adyen Executives & Org Chart - Clay,"Date\n\nNovember 15, 2024\n\nIndex\n\nTOC Head...",https://www.clay.com/dossier/adyen-executives,tavily,16,0.5,3
2,Adyen,CEO in 2024,Adyen - Wikipedia,| Company type | Naamloze vennootschap |\n| Tr...,https://en.wikipedia.org/wiki/Adyen,tavily,0,0.5,7
3,Adyen,CEO in 2024,Adyen - Wikipedia,| Company type | Naamloze vennootschap |\n| Tr...,https://en.wikipedia.org/wiki/Adyen,tavily,10,0.5,8
4,Adyen,CEO in 2024,Adyen N.V. (ADYENA) Leadership & Management Te...,Adyen N.V.\n\nBATS-CHIXE:ADYENA Stock Report\n...,https://simplywall.st/stocks/gb/diversified-fi...,tavily,1,0.5,11
...,...,...,...,...,...,...,...,...,...
1195,Zalando,revenue in 2024,Zalando's Double-Digit Growth Surge Through Se...,"By business model, Zalando concentrates more t...",https://www.modaes.com/global/companies/zaland...,tavily,13,0.5,3538
1196,Zalando,revenue in 2024,Zalando-SE_AGM-2025_1_Annual-report_2024.pdf,1 Company 2 Combined management report 3 Conso...,https://corporate.zalando.com/sites/default/fi...,tavily,6,0.5,3541
1197,Zalando,revenue in 2024,Zalando-SE_AGM-2025_1_Annual-report_2024.pdf,1 Company 2 Combined management report 3 Conso...,https://corporate.zalando.com/sites/default/fi...,tavily,16,0.5,3542
1198,Zalando,revenue in 2024,[PDF] Zalando group financials as of Q4 2024,Year FY/24 Average GMV per active customer (LT...,https://corporate.zalando.com/sites/default/fi...,tavily,8,1.0,3548


In [ ]:
# For the duplicated rows by columns 'company', 'query', 'title', 'snippet' (2 duplicates for each):
#    - For one, change 'engine' to 'you', for the other keep as it is.

# This is now performed on the df "matching" instead of "calculated_searches"
import numpy as np

# Identify duplicates in matching
dupes_mask = matching.duplicated(
    subset=["company", "query", "title", "snippet"], keep=False
)
dupes_df = matching[dupes_mask].copy()

# Enumerate within each group of duplicated rows
dupes_df["dupe_group_idx"] = dupes_df.groupby(
    ["company", "query", "title", "snippet"]
).cumcount()

# Assign engine="you" to one row in each duplicate group (for dupe_group_idx == 0)
rows_to_update = dupes_df["dupe_group_idx"] == 0
indices_to_update = dupes_df[rows_to_update].index

# Update 'engine' in the matching DataFrame
matching.loc[indices_to_update, "engine"] = "you"

# Clean up helper column if it somehow gets merged back
if "dupe_group_idx" in matching.columns:
    matching.drop(columns=["dupe_group_idx"], inplace=True, errors="ignore")


In [ ]:
calculated_searches_one_part = calculated_searches[
    calculated_searches["engine"] != "tavily"
]

In [90]:
all_searches = pd.concat([calculated_searches_one_part, matching])

In [ ]:
all_searches.drop(columns=["pos"], inplace=True, errors="ignore")
all_searches


,company,query,title,snippet,url,engine,search_num,score
0,Adyen,CEO in 2024,"""Legacy players will have a hard time"" — Adyen...","November 12, 2025 - The quote in the title com...",https://thedutchinvestors.substack.com/p/legac...,duckduckgo,8,1.0
1,Adyen,CEO in 2024,10 Adyen Executives & Org Chart - Clay,10 Adyen Executives &amp; Org Chart | Clay\n![...,https://www.clay.com/dossier/adyen-executives,exa,3,0.0
4,Adyen,CEO in 2024,10 Adyen Executives & Org Chart | Clay,Get an inside look at Adyen’s organizational c...,https://www.clay.com/dossier/adyen-executives,jina,5,0.5
5,Adyen,CEO in 2024,10 Adyen Executives & Org Chart | Clay,"His educational credentials, including a Maste...",https://www.clay.com/dossier/adyen-executives,brave,5,0.5
6,Adyen,CEO in 2024,Adyen - Wikipedia,1 month ago - Adyen was founded in 2006 by Pie...,https://en.wikipedia.org/wiki/Adyen,duckduckgo,2,0.5
...,...,...,...,...,...,...,...,...
1195,Zalando,revenue in 2024,Zalando's Double-Digit Growth Surge Through Se...,"By business model, Zalando concentrates more t...",https://www.modaes.com/global/companies/zaland...,tavily,13,0.5
1196,Zalando,revenue in 2024,Zalando-SE_AGM-2025_1_Annual-report_2024.pdf,1 Company 2 Combined management report 3 Conso...,https://corporate.zalando.com/sites/default/fi...,you,6,0.5
1197,Zalando,revenue in 2024,Zalando-SE_AGM-2025_1_Annual-report_2024.pdf,1 Company 2 Combined management report 3 Conso...,https://corporate.zalando.com/sites/default/fi...,tavily,16,0.5
1198,Zalando,revenue in 2024,[PDF] Zalando group financials as of Q4 2024,Year FY/24 Average GMV per active customer (LT...,https://corporate.zalando.com/sites/default/fi...,you,8,1.0


In [ ]:
all_searches["engine"].unique()

array(['duckduckgo', 'exa', 'jina', 'brave', 'you', 'tavily'],
      dtype=object)

In [ ]:
all_searches["search_num"] = all_searches.groupby(
    ["engine", "company", "query"]
).cumcount()


In [ ]:
all_searches.to_csv("data/searches_results_2.csv", index=False)

In [89]:
calculated_searches_one_part

,company,query,title,snippet,url,engine,search_num,score,pos
0,Adyen,CEO in 2024,"""Legacy players will have a hard time"" — Adyen...","November 12, 2025 - The quote in the title com...",https://thedutchinvestors.substack.com/p/legac...,duckduckgo,8,1.0,0
1,Adyen,CEO in 2024,10 Adyen Executives & Org Chart - Clay,10 Adyen Executives &amp; Org Chart | Clay\n![...,https://www.clay.com/dossier/adyen-executives,exa,3,0.0,1
4,Adyen,CEO in 2024,10 Adyen Executives & Org Chart | Clay,Get an inside look at Adyen’s organizational c...,https://www.clay.com/dossier/adyen-executives,jina,5,0.5,4
5,Adyen,CEO in 2024,10 Adyen Executives & Org Chart | Clay,"His educational credentials, including a Maste...",https://www.clay.com/dossier/adyen-executives,brave,5,0.5,5
6,Adyen,CEO in 2024,Adyen - Wikipedia,1 month ago - Adyen was founded in 2006 by Pie...,https://en.wikipedia.org/wiki/Adyen,duckduckgo,2,0.5,6
...,...,...,...,...,...,...,...,...,...
3545,Zalando,revenue in 2024,Zalando: y-o-y revenue growth 2024 | Statista,"As of the second quarter of 2024, Zalando&#x27...",https://www.statista.com/statistics/427245/zal...,brave,6,0.5,3545
3546,Zalando,revenue in 2024,Zalando’s Double-Digit Growth Surge Through Se...,"In the first three quarters, the German e-comm...",https://www.modaes.com/global/companies/zaland...,jina,6,0.5,3546
3547,Zalando,revenue in 2024,[PDF] Zalando group financials as of Q4 2024,1_Key figures\n2_Customer KPI development\n3_I...,https://corporate.zalando.com/sites/default/fi...,exa,5,1.0,3547
3550,Zalando,revenue in 2024,macrotrends.net/stocks/charts/ZLNDY/ zalando /...,Zalando revenue .,https://www.macrotrends.net/stocks/charts/ZLND...,duckduckgo,8,1.0,3550


In [88]:
matching

,company,query,title,snippet,url,engine,search_num,score,pos
0,Adyen,CEO in 2024,10 Adyen Executives & Org Chart - Clay,"Date\n\nNovember 15, 2024\n\nIndex\n\nTOC Head...",https://www.clay.com/dossier/adyen-executives,you,6,0.5,2
1,Adyen,CEO in 2024,10 Adyen Executives & Org Chart - Clay,"Date\n\nNovember 15, 2024\n\nIndex\n\nTOC Head...",https://www.clay.com/dossier/adyen-executives,tavily,16,0.5,3
2,Adyen,CEO in 2024,Adyen - Wikipedia,| Company type | Naamloze vennootschap |\n| Tr...,https://en.wikipedia.org/wiki/Adyen,you,0,0.5,7
3,Adyen,CEO in 2024,Adyen - Wikipedia,| Company type | Naamloze vennootschap |\n| Tr...,https://en.wikipedia.org/wiki/Adyen,tavily,10,0.5,8
4,Adyen,CEO in 2024,Adyen N.V. (ADYENA) Leadership & Management Te...,Adyen N.V.\n\nBATS-CHIXE:ADYENA Stock Report\n...,https://simplywall.st/stocks/gb/diversified-fi...,you,1,0.5,11
...,...,...,...,...,...,...,...,...,...
1195,Zalando,revenue in 2024,Zalando's Double-Digit Growth Surge Through Se...,"By business model, Zalando concentrates more t...",https://www.modaes.com/global/companies/zaland...,tavily,13,0.5,3538
1196,Zalando,revenue in 2024,Zalando-SE_AGM-2025_1_Annual-report_2024.pdf,1 Company 2 Combined management report 3 Conso...,https://corporate.zalando.com/sites/default/fi...,you,6,0.5,3541
1197,Zalando,revenue in 2024,Zalando-SE_AGM-2025_1_Annual-report_2024.pdf,1 Company 2 Combined management report 3 Conso...,https://corporate.zalando.com/sites/default/fi...,tavily,16,0.5,3542
1198,Zalando,revenue in 2024,[PDF] Zalando group financials as of Q4 2024,Year FY/24 Average GMV per active customer (LT...,https://corporate.zalando.com/sites/default/fi...,you,8,1.0,3548


In [ ]:
calculated_searches[calculated_searches["engine"].isin(positions)]

,company,query,title,snippet,url,engine,search_num,score,pos
2952,TotalEnergies,gross margin in 2024,[PDF] Results - TotalEnergies 3Q25,"equity (TotalEnergies share) 115,281 116,642 1...",https://totalenergies.com/system/files/documen...,tavily,8,0.5,2952
2953,TotalEnergies,gross margin in 2024,[PDF] Results - TotalEnergies 3Q25,"equity (TotalEnergies share) 115,281 116,642 1...",https://totalenergies.com/system/files/documen...,tavily,18,0.5,2953
2954,TotalEnergies,gross margin in 2024,[PDF] TotalEnergies PR 4Q23 - Results,"quarter 2024 versus $5,226 million in the four...",https://totalenergies.com/system/files/documen...,tavily,6,0.0,2954
2955,TotalEnergies,gross margin in 2024,[PDF] TotalEnergies PR 4Q23 - Results,"quarter 2024 versus $5,226 million in the four...",https://totalenergies.com/system/files/documen...,tavily,16,0.0,2955
2956,TotalEnergies,gross margin in 2024,NaN,NaN,https://www.alphaquery.com/stock/TTE/fundament...,jina,3,1.0,2956
...,...,...,...,...,...,...,...,...,...
3547,Zalando,revenue in 2024,[PDF] Zalando group financials as of Q4 2024,1_Key figures\n2_Customer KPI development\n3_I...,https://corporate.zalando.com/sites/default/fi...,exa,5,1.0,3547
3548,Zalando,revenue in 2024,[PDF] Zalando group financials as of Q4 2024,Year FY/24 Average GMV per active customer (LT...,https://corporate.zalando.com/sites/default/fi...,tavily,8,1.0,3548
3549,Zalando,revenue in 2024,[PDF] Zalando group financials as of Q4 2024,Year FY/24 Average GMV per active customer (LT...,https://corporate.zalando.com/sites/default/fi...,tavily,18,1.0,3549
3550,Zalando,revenue in 2024,macrotrends.net/stocks/charts/ZLNDY/ zalando /...,Zalando revenue .,https://www.macrotrends.net/stocks/charts/ZLND...,duckduckgo,8,1.0,3550


# Processing

In [ ]:
import pandas as pd
import asyncio
import json
from typing import List, Dict, Optional
import os
from datetime import datetime

# Define metrics mapping (metric name -> ground_truth column name)
METRICS_MAPPING = {
    "CEO in 2024": "ceo",
    "revenue in 2024": "revenue",
    "gross margin in 2024": "gross_margin",
}


# --- Process a single row ---
async def process_row(
    row: pd.Series,
    semaphore: asyncio.Semaphore,
    results_df: pd.DataFrame,
    results_file: str,
    row_index: int,
    save_lock: asyncio.Lock,
    save_counter: list,  # List to track count for periodic saving
):
    """Process a single row and update results incrementally."""
    async with semaphore:  # Limit concurrent requests
        try:
            # Get ground truth value
            query = row["query"]
            company = row["company"]
            gt_column = METRICS_MAPPING.get(query)
            gt_value = get_ground_truth_value(company, gt_column) if gt_column else None

            # Create prompt
            prompt = (
                f"Question: {query}. Real value {gt_value}. "
                f"Found document {row['title']} {row['snippet']}"
            )

            # Process with agent
            result = await information_test_agent.run(prompt)
            score = result.output.relevant_information

        except Exception as e:
            company = row.get("company", "unknown")
            query = row.get("query", "unknown")
            print(f"Error processing row {row_index} ({company} - {query}): {e}")
            score = None

        # Update the result in the dataframe (thread-safe)
        async with save_lock:
            results_df.at[row_index, "score"] = score
            save_counter[0] += 1

            # Save incrementally every 10 rows to avoid losing progress
            if save_counter[0] % 10 == 0:
                try:
                    results_df.to_csv(results_file, index=False)
                except Exception as e:
                    print(f"Error saving results: {e}")


# --- Main processing function ---
async def process_dataframe(
    df: pd.DataFrame,
    max_concurrent: int = 10,
    results_file: Optional[str] = None,
):
    """
    Process a DataFrame and add a 'score' column (0, 0.5, or 1).

    Parameters:
    -----------
    df : pd.DataFrame
        Input DataFrame with columns: company, query, title, snippet, url, engine, search_num
    max_concurrent : int
        Maximum number of concurrent requests (default: 10)
    results_file : str, optional
        Path to CSV file to save results incrementally. If None, auto-generates filename.

    Returns:
    --------
    pd.DataFrame with added 'score' column
    """
    # Create a copy to avoid modifying the original
    results_df = df.copy()
    results_df["score"] = None

    # Generate results filename if not provided
    if results_file is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        results_file = f"processing_results_{timestamp}.csv"

    print(
        f"Processing {len(results_df)} rows with max {max_concurrent} concurrent requests..."
    )
    print(f"Results will be saved to: {results_file}")

    # Create semaphore to limit concurrent requests
    semaphore = asyncio.Semaphore(max_concurrent)

    # Create lock for thread-safe saving
    save_lock = asyncio.Lock()
    save_counter = [0]  # Use list to allow modification in nested function

    # Create tasks for all rows
    tasks = []
    for idx, row in results_df.iterrows():
        task = process_row(
            row, semaphore, results_df, results_file, idx, save_lock, save_counter
        )
        tasks.append(task)

    # Execute all tasks
    await asyncio.gather(*tasks, return_exceptions=True)

    # Final save to ensure all results are saved
    try:
        results_df.to_csv(results_file, index=False)
        print(f"Processing complete! Results saved to: {results_file}")
    except Exception as e:
        print(f"Error in final save: {e}")

    return results_df


# --- Usage example ---
# results = await process_dataframe(all_searches, max_concurrent=10)
# print(results)

In [ ]:
calculated_results1 = pd.read_csv("processing_results_20260109_161727_1.csv")
calculated_results2 = pd.read_csv("processing_results_20260109_164203_exa.csv")
calculated_results3 = pd.read_csv("processing_results_20260109_164932.csv")
calculated_results4 = pd.read_csv("processing_results_20260109_165732.csv")

In [ ]:
def merge_dataframes_with_scores(
    df_list: List[pd.DataFrame], score_column: str = "score"
) -> pd.DataFrame:
    """
    Merge multiple dataframes, prioritizing non-NA values in the score column.

    Parameters:
    -----------
    df_list : List[pd.DataFrame]
        List of dataframes to merge. All should have the same structure.
    score_column : str
        Name of the score column to merge (default: 'score')

    Returns:
    --------
    pd.DataFrame
        Merged dataframe with non-NA scores prioritized
    """
    if not df_list:
        raise ValueError("df_list cannot be empty")

    if len(df_list) == 1:
        return df_list[0].copy()

    # Start with the first dataframe as base
    merged_df = df_list[0].copy()

    # Identify key columns for merging (all columns except score_column)
    key_columns = [col for col in merged_df.columns if col != score_column]

    # Merge with each subsequent dataframe
    for i, df in enumerate(df_list[1:], start=1):
        # Ensure the dataframe has the score column
        if score_column not in df.columns:
            print(
                f"Warning: DataFrame {i} does not have '{score_column}' column, skipping..."
            )
            continue

        # Merge on key columns
        merged_df = merged_df.merge(
            df[key_columns + [score_column]],
            on=key_columns,
            how="outer",  # Use outer to keep all rows
            suffixes=("", f"_{i}"),
        )

        # Fill NA scores from the current dataframe (prioritize non-NA values)
        score_col_new = f"{score_column}_{i}"
        if score_col_new in merged_df.columns:
            # Fill NA values in the original score column with values from the new column
            merged_df[score_column] = merged_df[score_column].fillna(
                merged_df[score_col_new]
            )
            # Drop the temporary column
            merged_df = merged_df.drop(columns=[score_col_new])

    return merged_df

In [ ]:
calculated_results = merge_dataframes_with_scores(
    [calculated_results1, calculated_results2, calculated_results3, calculated_results4]
)

In [ ]:
missing_results = calculated_results[calculated_results["score"].isna()]

In [36]:
missing_results

,company,query,title,snippet,url,engine,search_num,score


In [ ]:
results = await process_dataframe(missing_results, max_concurrent=30)

Processing 22 rows with max 30 concurrent requests...
Results will be saved to: processing_results_20260109_165732.csv
Processing complete! Results saved to: processing_results_20260109_165732.csv


In [ ]:
calculated_results.to_csv("searches_results.csv", index=False)

In [38]:
final_results = pd.read_csv("processing_results_20260109_165732.csv")

In [39]:
final_results

,company,query,title,snippet,url,engine,search_num,score
0,Zalando,CEO in 2024,Zalando SE (ZAL) Leadership & Management Team ...,NOT FOR DISTRIBUTION\n\nZalando SE\n\nWSE:ZAL ...,https://simplywall.st/stocks/pl/retail/wse-zal...,tavily,18,0.5
1,Zalando,CEO in 2024,Zalando-SE_AGM-2025_1_Annual-report_2024.pdf,to employees of the Zalando group and to membe...,https://corporate.zalando.com/sites/default/fi...,tavily,10,0.5
2,Zalando,CEO in 2024,[PDF] Explanatory statement to the remuneratio...,Explanatory statement to the remuneration repo...,https://corporate.zalando.com/sites/default/fi...,tavily,15,0.5
3,Zalando,gross margin in 2024,Annual Report 2024,Skip to cookie consent \n\nZalando Corporate W...,https://corporate.zalando.com/en/investor-rela...,tavily,19,0.0
4,Zalando,gross margin in 2024,Key figures 2024 - Zalando Corporate Website,| | 2024 | 2023 | Change |\n --- --- |\n| Ke...,https://corporate.zalando.com/en/investor-rela...,tavily,14,0.0
5,Zalando,gross margin in 2024,ZAL - Zalando SE Key Metrics - Morningstar,"| Income Statement (in Bil, except ratios) | ...",https://www.morningstar.com/stocks/xhan/zal/ke...,tavily,17,1.0
6,Zalando,gross margin in 2024,Zalando Gross Profit 2016-2025 | ZLNDY - Macro...,Macrotrends Logo\n\n## Zalando Gross Profit 20...,https://www.macrotrends.net/stocks/charts/ZLND...,tavily,13,0.5
7,Zalando,gross margin in 2024,Zalando SE Profit Margin | ZAL.DE - MLQ.ai,###### Operating Margin\n\n3.71%\n\n###### Gro...,https://mlq.ai/stocks/ZAL.DE/profit-margin/,tavily,12,1.0
8,Zalando,gross margin in 2024,Zalando Sees Accelerating Growth This Year Fol...,Driven by its focus on driving profitable grow...,https://corporate.zalando.com/en/investor-rela...,tavily,10,0.5
9,Zalando,gross margin in 2024,Zalando's 2024 results & 2025 growth strategy ...,Zalando’s 2024 results are solid and set the f...,https://ecommercegermany.com/blog/zalandos-202...,tavily,11,0.0
